In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Coronary_artery_disease/GSE115733'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Identification of circular RNA hsa_circ_001879 and hsa_circ_004104 as novel biomarkers for coronary artery disease"
!Series_summary	"The circRNA profiles of 24 CAD patients and 7 controls were performed by microarray. Six classifiers were used to evaluate potential circRNAs biomarkers. The expression levels of candidate circRNAs were further verified by qRT-PCR in large cohorts. Logistic regression analysis and receiver operating characteristic were conducted to assess the diagnostic value. A total of 624 circRNAs and 171 circRNAs were significantly upregulated and downregulated, respectively, in CAD patients relative to controls. A four-circRNA signature was identified using six different classifiers. Among these four circRNAs, hsa_circ_001879 and hsa_circ_004104 were found significantly upregulated in validation cohorts. We offered a transcriptome-wide overview of aberrantly expressed circRNAs in CAD patients, and Hsa_circ_001879 and hsa_circ_00

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Determine if gene expression data is available
# Given dataset has circRNA data, so gene expression data is not available.
is_gene_available = False

# Check for 'Coronary_artery_disease'
trait_row = None
if "diagnosis" in next((v for k, v in {0: ['diagnosis: coronary artery disease', 'diagnosis: Healthy control']}.items()), []):
    trait_row = 0

# Check for 'age'
age_row = None
# Not mentioned in the sample characteristics

# Check for 'gender'
gender_row = None
if "gender" in next((v for k, v in {2: ['gender: male']}.items()), []):
    gender_row = 2

# Define conversion functions
def convert_trait(value):
    if "coronary artery disease" in value.lower():
        return 1
    elif "healthy control" in value.lower():
        return 0
    return None

# No age data available, skip defining convert_age
convert_age = None

def convert_gender(value):
    val = value.split(":")[-1].strip().lower()
    if val == "male":
        return 1
    elif val == "female":
        return 0
    return None

# Save cohort information
save_cohort_info('GSE115733', './preprocessed/Coronary_artery_disease/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Coronary_artery_disease', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Coronary_artery_disease/trait_data/GSE115733.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


A new JSON file was created at: ./preprocessed/Coronary_artery_disease/cohort_info.json
